# The Stigler diet problem

Problem summary: Satisfying recommended nutrient intakes with the least amount of money.

Problem details: [see here](https://en.wikipedia.org/wiki/Stigler_diet)

The purpose of this notebook is to try out Google's [OR-Tools](https://developers.google.com/optimization).

The exercise below was taken from [here](https://developers.google.com/optimization/lp/glop).

In [1]:
from ortools.linear_solver import pywraplp
import pandas as pd
from loguru import logger

## Define data

The following values are nutrients per dollar

In [2]:
food_data_col = ['Commodity', 'Unit', '1939 price (cents)', 'Calories', 'Protein (g)', 'Calcium (g)', 'Iron (mg)',
'Vitamin A (IU)', 'Thiamine (mg)', 'Riboflavin (mg)', 'Niacin (mg)', 'Ascorbic Acid (mg)']

food_data = [
['Wheat Flour (Enriched)', '10 lb.', 36, 44.7, 1411, 2, 365, 0, 55.4, 33.3, 441, 0],
['Macaroni', '1 lb.', 14.1, 11.6, 418, 0.7, 54, 0, 3.2, 1.9, 68, 0],
['Wheat Cereal (Enriched)', '28 oz.', 24.2, 11.8, 377, 14.4, 175, 0, 14.4, 8.8, 114, 0],
['Corn Flakes', '8 oz.', 7.1, 11.4, 252, 0.1, 56, 0, 13.5, 2.3, 68, 0],
['Corn Meal', '1 lb.', 4.6, 36.0, 897, 1.7, 99, 30.9, 17.4, 7.9, 106, 0],
['Hominy Grits', '24 oz.', 8.5, 28.6, 680, 0.8, 80, 0, 10.6, 1.6, 110, 0],
['Rice', '1 lb.', 7.5, 21.2, 460, 0.6, 41, 0, 2, 4.8, 60, 0],
['Rolled Oats', '1 lb.', 7.1, 25.3, 907, 5.1, 341, 0, 37.1, 8.9, 64, 0],
['White Bread (Enriched)', '1 lb.', 7.9, 15.0, 488, 2.5, 115, 0, 13.8, 8.5, 126, 0],
['Whole Wheat Bread', '1 lb.', 9.1, 12.2, 484, 2.7, 125, 0, 13.9, 6.4, 160, 0],
['Rye Bread', '1 lb.', 9.1, 12.4, 439, 1.1, 82, 0, 9.9, 3, 66, 0],
['Pound Cake', '1 lb.', 24.8, 8.0, 130, 0.4, 31, 18.9, 2.8, 3, 17, 0],
['Soda Crackers', '1 lb.', 15.1, 12.5, 288, 0.5, 50, 0, 0, 0, 0, 0],
['Milk', '1 qt.', 11, 6.1, 310, 10.5, 18, 16.8, 4, 16, 7, 177],
['Evaporated Milk (can)', '14.5 oz.', 6.7, 8.4, 422, 15.1, 9, 26, 3, 23.5, 11, 60],
['Butter', '1 lb.', 30.8, 10.8, 9, 0.2, 3, 44.2, 0, 0.2, 2, 0],
['Oleomargarine', '1 lb.', 16.1, 20.6, 17, 0.6, 6, 55.8, 0.2, 0, 0, 0],
['Eggs', '1 doz.', 32.6, 2.9, 238, 1.0, 52, 18.6, 2.8, 6.5, 1, 0],
['Cheese (Cheddar)', '1 lb.', 24.2, 7.4, 448, 16.4, 19, 28.1, 0.8, 10.3, 4, 0],
['Cream', '1/2 pt.', 14.1, 3.5, 49, 1.7, 3, 16.9, 0.6, 2.5, 0, 17],
['Peanut Butter', '1 lb.', 17.9, 15.7, 661, 1.0, 48, 0, 9.6, 8.1, 471, 0],
['Mayonnaise', '1/2 pt.', 16.7, 8.6, 18, 0.2, 8, 2.7, 0.4, 0.5, 0, 0],
['Crisco', '1 lb.', 20.3, 20.1, 0, 0, 0, 0, 0, 0, 0, 0],
['Lard', '1 lb.', 9.8, 41.7, 0, 0, 0, 0.2, 0, 0.5, 5, 0],
['Sirloin Steak', '1 lb.', 39.6, 2.9, 166, 0.1, 34, 0.2, 2.1, 2.9, 69, 0],
['Round Steak', '1 lb.', 36.4, 2.2, 214, 0.1, 32, 0.4, 2.5, 2.4, 87, 0],
['Rib Roast', '1 lb.', 29.2, 3.4, 213, 0.1, 33, 0, 0, 2, 0, 0],
['Chuck Roast', '1 lb.', 22.6, 3.6, 309, 0.2, 46, 0.4, 1, 4, 120, 0],
['Plate', '1 lb.', 14.6, 8.5, 404, 0.2, 62, 0, 0.9, 0, 0, 0],
['Liver (Beef)', '1 lb.', 26.8, 2.2, 333, 0.2, 139, 169.2, 6.4, 50.8, 316, 525],
['Leg of Lamb', '1 lb.', 27.6, 3.1, 245, 0.1, 20, 0, 2.8, 3.9, 86, 0],
['Lamb Chops (Rib)', '1 lb.', 36.6, 3.3, 140, 0.1, 15, 0, 1.7, 2.7, 54, 0],
['Pork Chops', '1 lb.', 30.7, 3.5, 196, 0.2, 30, 0, 17.4, 2.7, 60, 0],
['Pork Loin Roast', '1 lb.', 24.2, 4.4, 249, 0.3, 37, 0, 18.2, 3.6, 79, 0],
['Bacon', '1 lb.', 25.6, 10.4, 152, 0.2, 23, 0, 1.8, 1.8, 71, 0],
['Ham, smoked', '1 lb.', 27.4, 6.7, 212, 0.2, 31, 0, 9.9, 3.3, 50, 0],
['Salt Pork', '1 lb.', 16, 18.8, 164, 0.1, 26, 0, 1.4, 1.8, 0, 0],
['Roasting Chicken', '1 lb.', 30.3, 1.8, 184, 0.1, 30, 0.1, 0.9, 1.8, 68, 46],
['Veal Cutlets', '1 lb.', 42.3, 1.7, 156, 0.1, 24, 0, 1.4, 2.4, 57, 0],
['Salmon, Pink (can)', '16 oz.', 13, 5.8, 705, 6.8, 45, 3.5, 1, 4.9, 209, 0],
['Apples', '1 lb.', 4.4, 5.8, 27, 0.5, 36, 7.3, 3.6, 2.7, 5, 544],
['Bananas', '1 lb.', 6.1, 4.9, 60, 0.4, 30, 17.4, 2.5, 3.5, 28, 498],
['Lemons', '1 doz.', 26, 1.0, 21, 0.5, 14, 0, 0.5, 0, 4, 952],
['Oranges', '1 doz.', 30.9, 2.2, 40, 1.1, 18, 11.1, 3.6, 1.3, 10, 1998],
['Green Beans', '1 lb.', 7.1, 2.4, 138, 3.7, 80, 69, 4.3, 5.8, 37, 862],
['Cabbage', '1 lb.', 3.7, 2.6, 125, 4.0, 36, 7.2, 9, 4.5, 26, 5369],
['Carrots', '1 bunch', 4.7, 2.7, 73, 2.8, 43, 188.5, 6.1, 4.3, 89, 608],
['Celery', '1 stalk', 7.3, 0.9, 51, 3.0, 23, 0.9, 1.4, 1.4, 9, 313],
['Lettuce', '1 head', 8.2, 0.4, 27, 1.1, 22, 112.4, 1.8, 3.4, 11, 449],
['Onions', '1 lb.', 3.6, 5.8, 166, 3.8, 59, 16.6, 4.7, 5.9, 21, 1184],
['Potatoes', '15 lb.', 34, 14.3, 336, 1.8, 118, 6.7, 29.4, 7.1, 198, 2522],
['Spinach', '1 lb.', 8.1, 1.1, 106, 0, 138, 918.4, 5.7, 13.8, 33, 2755],
['Sweet Potatoes', '1 lb.', 5.1, 9.6, 138, 2.7, 54, 290.7, 8.4, 5.4, 83, 1912],
['Peaches (can)', 'No. 2 1/2', 16.8, 3.7, 20, 0.4, 10, 21.5, 0.5, 1, 31, 196],
['Pears (can)', 'No. 2 1/2', 20.4, 3.0, 8, 0.3, 8, 0.8, 0.8, 0.8, 5, 81],
['Pineapple (can)', 'No. 2 1/2', 21.3, 2.4, 16, 0.4, 8, 2, 2.8, 0.8, 7, 399],
['Asparagus (can)', 'No. 2', 27.7, 0.4, 33, 0.3, 12, 16.3, 1.4, 2.1, 17, 272],
['Green Beans (can)', 'No. 2', 10, 1.0, 54, 2, 65, 53.9, 1.6, 4.3, 32, 431],
['Pork and Beans (can)', '16 oz.', 7.1, 7.5, 364, 4, 134, 3.5, 8.3, 7.7, 56, 0],
['Corn (can)', 'No. 2', 10.4, 5.2, 136, 0.2, 16, 12, 1.6, 2.7, 42, 218],
['Peas (can)', 'No. 2', 13.8, 2.3, 136, 0.6, 45, 34.9, 4.9, 2.5, 37, 370],
['Tomatoes (can)', 'No. 2', 8.6, 1.3, 63, 0.7, 38, 53.2, 3.4, 2.5, 36, 1253],
['Tomato Soup (can)', '10 1/2 oz.', 7.6, 1.6, 71, 0.6, 43, 57.9, 3.5, 2.4, 67, 862],
['Peaches, Dried', '1 lb.', 15.7, 8.5, 87, 1.7, 173, 86.8, 1.2, 4.3, 55, 57],
['Prunes, Dried', '1 lb.', 9, 12.8, 99, 2.5, 154, 85.7, 3.9, 4.3, 65, 257],
['Raisins, Dried', '15 oz.', 9.4, 13.5, 104, 2.5, 136, 4.5, 6.3, 1.4, 24, 136],
['Peas, Dried', '1 lb.', 7.9, 20.0, 1367, 4.2, 345, 2.9, 28.7, 18.4, 162, 0],
['Lima Beans, Dried', '1 lb.', 8.9, 17.4, 1055, 3.7, 459, 5.1, 26.9, 38.2, 93, 0],
['Navy Beans, Dried', '1 lb.', 5.9, 26.9, 1691, 11.4, 792, 0, 38.4, 24.6, 217, 0],
['Coffee', '1 lb.', 22.4, 0, 0, 0, 0, 0, 4, 5.1, 50, 0],
['Tea', '1/4 lb.', 17.4, 0, 0, 0, 0, 0, 0, 2.3, 42, 0],
['Cocoa', '8 oz.', 8.6, 8.7, 237, 3, 72, 0, 2, 11.9, 40, 0],
['Chocolate', '8 oz.', 16.2, 8.0, 77, 1.3, 39, 0, 0.9, 3.4, 14, 0],
['Sugar', '10 lb.', 51.7, 34.9, 0, 0, 0, 0, 0, 0, 0, 0],
['Corn Syrup', '24 oz.', 13.7, 14.7, 0, 0.5, 74, 0, 0, 0, 5, 0],
['Molasses', '18 oz.', 13.6, 9.0, 0, 10.3, 244, 0, 1.9, 7.5, 146, 0],
['Strawberry Preserves', '1 lb.', 20.5, 6.4, 11, 0.4, 7, 0.2, 0.2, 0.4, 3, 0]];

# Nutrient minimums.
nutrient_col = ['Nutrient', 'Daily Recommended Intake']
nutrients = [
  ['Calories (1000s)', 3],
  ['Protein (grams)', 70],
  ['Calcium (grams)', 0.8],
  ['Iron (mg)', 12],
  ['Vitamin A (1000 IU)', 5],
  ['Vitamin B1 (mg)', 1.8],
  ['Vitamin B2 (mg)', 2.7],
  ['Niacin (mg)', 18],
  ['Vitamin C (mg)', 75]]

In [3]:
pd.DataFrame(food_data, columns=food_data_col)

,Commodity,Unit,1939 price (cents),Calories,Protein (g),Calcium (g),Iron (mg),Vitamin A (IU),Thiamine (mg),Riboflavin (mg),Niacin (mg),Ascorbic Acid (mg)
0,Wheat Flour (Enriched),10 lb.,36.0,44.7,1411,2.0,365,0.0,55.4,33.3,441,0
1,Macaroni,1 lb.,14.1,11.6,418,0.7,54,0.0,3.2,1.9,68,0
2,Wheat Cereal (Enriched),28 oz.,24.2,11.8,377,14.4,175,0.0,14.4,8.8,114,0
3,Corn Flakes,8 oz.,7.1,11.4,252,0.1,56,0.0,13.5,2.3,68,0
4,Corn Meal,1 lb.,4.6,36.0,897,1.7,99,30.9,17.4,7.9,106,0
5,Hominy Grits,24 oz.,8.5,28.6,680,0.8,80,0.0,10.6,1.6,110,0
6,Rice,1 lb.,7.5,21.2,460,0.6,41,0.0,2.0,4.8,60,0
7,Rolled Oats,1 lb.,7.1,25.3,907,5.1,341,0.0,37.1,8.9,64,0
8,White Bread (Enriched),1 lb.,7.9,15.0,488,2.5,115,0.0,13.8,8.5,126,0
9,Whole Wheat Bread,1 lb.,9.1,12.2,484,2.7,125,0.0,13.9,6.4,160,0


In [4]:
pd.DataFrame(nutrients, columns=nutrient_col)

,Nutrient,Daily Recommended Intake
0,Calories (1000s),3.0
1,Protein (grams),70.0
2,Calcium (grams),0.8
3,Iron (mg),12.0
4,Vitamin A (1000 IU),5.0
5,Vitamin B1 (mg),1.8
6,Vitamin B2 (mg),2.7
7,Niacin (mg),18.0
8,Vitamin C (mg),75.0


In [5]:
print(f'`food_data` length: {len(food_data)}')
print(f'`nutrients` length: {len(nutrients)}')

`food_data` length: 77
`nutrients` length: 9


## Declare solver

GLOP is Google's linear programming system.

In [6]:
# Instantiate a Glop solver, naming it SolveStigler.
solver = pywraplp.Solver('SolveStigler',
                       pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

# number of milliseconds
solver.SetTimeLimit(60*100)

## Set objective: minimize the sum of (price-normalized) foods.

Objective: $min \ P_{wheat\_flour} + P_{macaroni} + P_{wheat\_cereal} + ... $

In [7]:
# Decision variable: 1price[i] is the amount of money to spend on foodstuff i
price = [[]] * len(food_data)

objective = solver.Objective()
for i in range(0, len(food_data)):
    # Declare continuous variables (lb, ub, name)
    price[i] = solver.NumVar(0.0, solver.infinity(), food_data[i][0])
    
    # Set coefficient of each variable in the objective
    # There is no coefficient for any of the terms, so set to 1
    objective.SetCoefficient(price[i], 1)

# declares to be a minimization problem
objective.SetMinimization()

## Create the constraints, one per nutrient.

Amount of nutrients must be greater than or equal to the values specified in `nutrients`

In [8]:
constraints = [0] * len(nutrients)
for i in range(0, len(nutrients)):
    logger.debug(f'i: {i} ({nutrients[i][0]})')
    # boundaries of contraints (lb, ub)
    # we want >= the amount specified in `nutrients` -> lb=nutrients[i][1]
    # assume there is no limit on ub, but in reality there are probably constraints
    constraints[i] = solver.Constraint(nutrients[i][1], solver.infinity())
    
    for j in range(0, len(food_data)):
        logger.debug(f'  j: {j} ({price[j]})')
        # dollar * nutrient per dollar = amount of nutrients
        constraints[i].SetCoefficient(price[j], food_data[j][i+3])

2020-02-11 20:05:12.400 | DEBUG    | __main__:<module>:3 - i: 0 (Calories (1000s))
2020-02-11 20:05:12.403 | DEBUG    | __main__:<module>:10 -   j: 0 (Wheat Flour (Enriched))
2020-02-11 20:05:12.404 | DEBUG    | __main__:<module>:10 -   j: 1 (Macaroni)
2020-02-11 20:05:12.406 | DEBUG    | __main__:<module>:10 -   j: 2 (Wheat Cereal (Enriched))
2020-02-11 20:05:12.407 | DEBUG    | __main__:<module>:10 -   j: 3 (Corn Flakes)
2020-02-11 20:05:12.410 | DEBUG    | __main__:<module>:10 -   j: 4 (Corn Meal)
2020-02-11 20:05:12.411 | DEBUG    | __main__:<module>:10 -   j: 5 (Hominy Grits)
2020-02-11 20:05:12.412 | DEBUG    | __main__:<module>:10 -   j: 6 (Rice)
2020-02-11 20:05:12.414 | DEBUG    | __main__:<module>:10 -   j: 7 (Rolled Oats)
2020-02-11 20:05:12.415 | DEBUG    | __main__:<module>:10 -   j: 8 (White Bread (Enriched))
2020-02-11 20:05:12.416 | DEBUG    | __main__:<module>:10 -   j: 9 (Whole Wheat Bread)
2020-02-11 20:05:12.420 | DEBUG    | __main__:<module>:10 -   j: 10 (Rye Bread

2020-02-11 20:05:12.538 | DEBUG    | __main__:<module>:10 -   j: 21 (Mayonnaise)
2020-02-11 20:05:12.538 | DEBUG    | __main__:<module>:10 -   j: 22 (Crisco)
2020-02-11 20:05:12.541 | DEBUG    | __main__:<module>:10 -   j: 23 (Lard)
2020-02-11 20:05:12.542 | DEBUG    | __main__:<module>:10 -   j: 24 (Sirloin Steak)
2020-02-11 20:05:12.542 | DEBUG    | __main__:<module>:10 -   j: 25 (Round Steak)
2020-02-11 20:05:12.543 | DEBUG    | __main__:<module>:10 -   j: 26 (Rib Roast)
2020-02-11 20:05:12.544 | DEBUG    | __main__:<module>:10 -   j: 27 (Chuck Roast)
2020-02-11 20:05:12.546 | DEBUG    | __main__:<module>:10 -   j: 28 (Plate)
2020-02-11 20:05:12.546 | DEBUG    | __main__:<module>:10 -   j: 29 (Liver (Beef))
2020-02-11 20:05:12.547 | DEBUG    | __main__:<module>:10 -   j: 30 (Leg of Lamb)
2020-02-11 20:05:12.548 | DEBUG    | __main__:<module>:10 -   j: 31 (Lamb Chops (Rib))
2020-02-11 20:05:12.549 | DEBUG    | __main__:<module>:10 -   j: 32 (Pork Chops)
2020-02-11 20:05:12.550 | DEBU

2020-02-11 20:05:12.643 | DEBUG    | __main__:<module>:10 -   j: 44 (Green Beans)
2020-02-11 20:05:12.645 | DEBUG    | __main__:<module>:10 -   j: 45 (Cabbage)
2020-02-11 20:05:12.646 | DEBUG    | __main__:<module>:10 -   j: 46 (Carrots)
2020-02-11 20:05:12.647 | DEBUG    | __main__:<module>:10 -   j: 47 (Celery)
2020-02-11 20:05:12.648 | DEBUG    | __main__:<module>:10 -   j: 48 (Lettuce)
2020-02-11 20:05:12.650 | DEBUG    | __main__:<module>:10 -   j: 49 (Onions)
2020-02-11 20:05:12.651 | DEBUG    | __main__:<module>:10 -   j: 50 (Potatoes)
2020-02-11 20:05:12.652 | DEBUG    | __main__:<module>:10 -   j: 51 (Spinach)
2020-02-11 20:05:12.652 | DEBUG    | __main__:<module>:10 -   j: 52 (Sweet Potatoes)
2020-02-11 20:05:12.653 | DEBUG    | __main__:<module>:10 -   j: 53 (Peaches (can))
2020-02-11 20:05:12.654 | DEBUG    | __main__:<module>:10 -   j: 54 (Pears (can))
2020-02-11 20:05:12.655 | DEBUG    | __main__:<module>:10 -   j: 55 (Pineapple (can))
2020-02-11 20:05:12.655 | DEBUG    |

2020-02-11 20:05:12.760 | DEBUG    | __main__:<module>:10 -   j: 66 (Peas, Dried)
2020-02-11 20:05:12.761 | DEBUG    | __main__:<module>:10 -   j: 67 (Lima Beans, Dried)
2020-02-11 20:05:12.762 | DEBUG    | __main__:<module>:10 -   j: 68 (Navy Beans, Dried)
2020-02-11 20:05:12.763 | DEBUG    | __main__:<module>:10 -   j: 69 (Coffee)
2020-02-11 20:05:12.764 | DEBUG    | __main__:<module>:10 -   j: 70 (Tea)
2020-02-11 20:05:12.765 | DEBUG    | __main__:<module>:10 -   j: 71 (Cocoa)
2020-02-11 20:05:12.766 | DEBUG    | __main__:<module>:10 -   j: 72 (Chocolate)
2020-02-11 20:05:12.767 | DEBUG    | __main__:<module>:10 -   j: 73 (Sugar)
2020-02-11 20:05:12.769 | DEBUG    | __main__:<module>:10 -   j: 74 (Corn Syrup)
2020-02-11 20:05:12.770 | DEBUG    | __main__:<module>:10 -   j: 75 (Molasses)
2020-02-11 20:05:12.771 | DEBUG    | __main__:<module>:10 -   j: 76 (Strawberry Preserves)
2020-02-11 20:05:12.772 | DEBUG    | __main__:<module>:3 - i: 4 (Vitamin A (1000 IU))
2020-02-11 20:05:12.77

2020-02-11 20:05:12.863 | DEBUG    | __main__:<module>:10 -   j: 10 (Rye Bread)
2020-02-11 20:05:12.864 | DEBUG    | __main__:<module>:10 -   j: 11 (Pound Cake)
2020-02-11 20:05:12.865 | DEBUG    | __main__:<module>:10 -   j: 12 (Soda Crackers)
2020-02-11 20:05:12.866 | DEBUG    | __main__:<module>:10 -   j: 13 (Milk)
2020-02-11 20:05:12.867 | DEBUG    | __main__:<module>:10 -   j: 14 (Evaporated Milk (can))
2020-02-11 20:05:12.868 | DEBUG    | __main__:<module>:10 -   j: 15 (Butter)
2020-02-11 20:05:12.869 | DEBUG    | __main__:<module>:10 -   j: 16 (Oleomargarine)
2020-02-11 20:05:12.870 | DEBUG    | __main__:<module>:10 -   j: 17 (Eggs)
2020-02-11 20:05:12.870 | DEBUG    | __main__:<module>:10 -   j: 18 (Cheese (Cheddar))
2020-02-11 20:05:12.871 | DEBUG    | __main__:<module>:10 -   j: 19 (Cream)
2020-02-11 20:05:12.873 | DEBUG    | __main__:<module>:10 -   j: 20 (Peanut Butter)
2020-02-11 20:05:12.874 | DEBUG    | __main__:<module>:10 -   j: 21 (Mayonnaise)
2020-02-11 20:05:12.874 

2020-02-11 20:05:12.965 | DEBUG    | __main__:<module>:10 -   j: 33 (Pork Loin Roast)
2020-02-11 20:05:12.966 | DEBUG    | __main__:<module>:10 -   j: 34 (Bacon)
2020-02-11 20:05:12.967 | DEBUG    | __main__:<module>:10 -   j: 35 (Ham, smoked)
2020-02-11 20:05:12.968 | DEBUG    | __main__:<module>:10 -   j: 36 (Salt Pork)
2020-02-11 20:05:12.969 | DEBUG    | __main__:<module>:10 -   j: 37 (Roasting Chicken)
2020-02-11 20:05:12.970 | DEBUG    | __main__:<module>:10 -   j: 38 (Veal Cutlets)
2020-02-11 20:05:12.971 | DEBUG    | __main__:<module>:10 -   j: 39 (Salmon, Pink (can))
2020-02-11 20:05:12.972 | DEBUG    | __main__:<module>:10 -   j: 40 (Apples)
2020-02-11 20:05:12.973 | DEBUG    | __main__:<module>:10 -   j: 41 (Bananas)
2020-02-11 20:05:12.974 | DEBUG    | __main__:<module>:10 -   j: 42 (Lemons)
2020-02-11 20:05:12.975 | DEBUG    | __main__:<module>:10 -   j: 43 (Oranges)
2020-02-11 20:05:12.975 | DEBUG    | __main__:<module>:10 -   j: 44 (Green Beans)
2020-02-11 20:05:12.976 |

2020-02-11 20:05:13.052 | DEBUG    | __main__:<module>:10 -   j: 56 (Asparagus (can))
2020-02-11 20:05:13.053 | DEBUG    | __main__:<module>:10 -   j: 57 (Green Beans (can))
2020-02-11 20:05:13.054 | DEBUG    | __main__:<module>:10 -   j: 58 (Pork and Beans (can))
2020-02-11 20:05:13.054 | DEBUG    | __main__:<module>:10 -   j: 59 (Corn (can))
2020-02-11 20:05:13.055 | DEBUG    | __main__:<module>:10 -   j: 60 (Peas (can))
2020-02-11 20:05:13.056 | DEBUG    | __main__:<module>:10 -   j: 61 (Tomatoes (can))
2020-02-11 20:05:13.057 | DEBUG    | __main__:<module>:10 -   j: 62 (Tomato Soup (can))
2020-02-11 20:05:13.058 | DEBUG    | __main__:<module>:10 -   j: 63 (Peaches, Dried)
2020-02-11 20:05:13.059 | DEBUG    | __main__:<module>:10 -   j: 64 (Prunes, Dried)
2020-02-11 20:05:13.060 | DEBUG    | __main__:<module>:10 -   j: 65 (Raisins, Dried)
2020-02-11 20:05:13.061 | DEBUG    | __main__:<module>:10 -   j: 66 (Peas, Dried)
2020-02-11 20:05:13.062 | DEBUG    | __main__:<module>:10 -   j:

## Solve

In [9]:
status = solver.Solve()

## Display solution

In [10]:
if status == solver.OPTIMAL:
    # Display the amounts (in dollars) to purchase of each food.
    price_total = 0
    num_nutrients = len(food_data[i]) - 3
    nutrients = [0] * num_nutrients
    for i in range(0, len(food_data)):
        price_total += price[i].solution_value()

        for nutrient in range(0, num_nutrients):
            nutrients[nutrient] += food_data[i][nutrient+3] * price[i].solution_value()

        if price[i].solution_value() > 0:
            print('%s = %f' % (food_data[i][0], price[i].solution_value()))

    print('Optimal annual price: $%.2f' % (365 * price_total))
else:  # No optimal solution was found.
    if status == solver.FEASIBLE:
        print('A potentially suboptimal solution was found.')
    else:
        print('The solver could not solve the problem.')

Wheat Flour (Enriched) = 0.029519
Liver (Beef) = 0.001893
Cabbage = 0.011214
Spinach = 0.005008
Navy Beans, Dried = 0.061029
Optimal annual price: $39.66
